<a href="https://colab.research.google.com/github/Ananya-AJ/CMPE255-Data-Mining/blob/main/Apache_Beam/Apachebeam_GroceryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A data pipeline using APACHE BEAM, Use the table of contents to navigate to a particular cell


##Install Apache beam and restart runtime

In [ ]:
! pip install apache-beam[interactive]


In [ ]:
!pip install apache_beam[dataframe]

In [ ]:
import apache_beam as beam
import pandas as pd
from apache_beam.dataframe.io import read_csv

##get a text file from git

In [ ]:
import requests

#master = "https://raw.githubusercontent.com/vigneshSs-07/Cloud-AI-Analytics/main/Apache Beam -Python/data/grocery.txt"
#master = "https://raw.githubusercontent.com/vigneshSs-07/Cloud-AI-Analytics/main/Apache%20Beam%20-Python/data/dept_data.txt"
req = requests.get(master)
req = req.text
with open('/content/dept.txt','w') as fp:
  fp.write(req)


##Transform definitions

In [ ]:
import apache_beam as beam

def SplitRow(element):
  return element.split(',')
  
def format_output(element):
  name, count = element
  return (name.encode('ascii'),str(count),'Avg priced')


##Instantiate the pipeline

In [ ]:
pipe = beam.Pipeline()

##Build a custom transform class

In [ ]:
class CusTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    final_op = ( 
        input_coll
         
                      |'to get the average of the elements for each unique key'>>beam.combiners.Mean.PerKey()
                      |'fixed-size random samples for each unique key in a PCollection'>>beam.combiners.Sample.FixedSizePerKey(3 or 0)
                      | 'Items and their location' >> beam.Map(format_output)
              
    )
    return final_op

##Pass the input data, Get rows that have item_type as dairy and baking goods and then call CusTransform for both the datasets, These filtered rows based on item_type is passed as pcollection to the custome transformation that runs all the composite transformations.

In [ ]:
with beam.Pipeline() as p:
  input_data = (p 
                      | "Read from text file" >> beam.io.ReadFromText('/content/grocery.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )
  
  Dairy_count = (
                      input_data
                      | 'Get all dairy item types' >> beam.Filter(lambda record: record[4] == 'Dairy')
                      | 'Pair each dairy item types with 1' >> beam.Map(lambda record: (record[1], 1))
                      | 'composite Dairy' >> CusTransform()
                      | 'Write results for Dairy' >> beam.io.WriteToText('/content/Dairy')
                 )

  Bakinggoods_count = (
                  input_data
                  | 'Get all baking goods item types' >> beam.Filter(lambda record: record[4] == 'Baking Goods')
                  | 'Pair each baking goods item type with 1' >> beam.Map(lambda record: (record[1], 1))
                  | 'composite Bakinggoods' >> CusTransform()
                  | 'Write results for Bakinggoods' >> beam.io.WriteToText('/content/Bakinggoods')
            )

In [ ]:
!{('head  /content/Bakinggoods-00000-of-00001')}

(b'10.395', '[1.0]', 'Avg priced')
(b'', '[1.0]', 'Avg priced')
(b'15.85', '[1.0]', 'Avg priced')


In [ ]:
!{('head  /content/Dairy-00000-of-00001')}

(b'9.3', '[1.0]', 'Avg priced')
(b'18.5', '[1.0]', 'Avg priced')
(b'13.35', '[1.0]', 'Avg priced')
(b'5.925', '[1.0]', 'Avg priced')
(b'19.25', '[1.0]', 'Avg priced')
(b'6.42', '[1.0]', 'Avg priced')
